In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
df=pd.read_csv('./Train.csv')
df_test=pd.read_csv('./Test.csv')

df['target']=df['target'].apply(lambda x:0 if x=='negative' else 1 )
df=df.drop(['Index'],axis=1)

df_test=df_test.drop(['Index'],axis=1)

In [3]:
y_origin=df['target']
x_origin=df.drop(['target'],axis=1)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
std_df_train = scaler.fit_transform(x_origin)
std_df_test2 = scaler.transform(df_test)

std_df_train = pd.DataFrame(std_df_train, index=x_origin.index, columns=x_origin.columns)
std_df_test2 = pd.DataFrame(std_df_test2, index=df_test.index, columns=df_test.columns)

In [5]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks#오버샘플링

smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'),random_state=112)
X_train_over, Y_train_over = smoteto.fit_sample(std_df_train, y_origin)

In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=240,random_state=112)
pca_train = pca.fit_transform(X_train_over)
pca_train = pd.DataFrame(pca_train, index=X_train_over.index,columns=[f"pca{num+1}" for num in range(0,240)])
pca_test2= pca.transform(std_df_test2)
pca_test2 = pd.DataFrame(pca_test2, index=df_test.index,columns=[f"pca{num+1}" for num in range(0,240)])

In [7]:
from sklearn.preprocessing import MinMaxScaler 

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(pca_train) 
X_test2_scaled= scaler.transform(pca_test2)

In [8]:
X_train_=pd.DataFrame(X_train_scaled,columns=pca_train.columns)
X_test2=pd.DataFrame(X_test2_scaled,columns=pca_test2.columns)

In [9]:
svc = SVC(kernel='rbf',C = 0.7, gamma = 0.5, probability=True)

In [10]:
knn = KNeighborsClassifier(n_neighbors = 2,weights='distance',n_jobs=-1)

In [11]:
poly_svc =SVC(kernel='poly',degree=20,C = 0.4, gamma = 0.2,max_iter=1000,probability=True)

In [12]:
sgd = SGDClassifier(loss='log', max_iter=100000,random_state=100) 

In [13]:
lgbm=LGBMClassifier(n_estimators=1000,max_depth=4,num_leaves=10,n_jobs=1,application='binary',objective='binary')

In [14]:
Xgb=XGBClassifier(objective='binary:logistic',colsample_bytree= 0.8,learning_rate= .1,max_depth= 5,min_child_weight=2,reg_lambda=10,reg_alpha=10,gamma=0.2,n_estimator=1000)

In [15]:
models = [
    ('svm', svc),
    ('poly',poly_svc)
    ,('Xgb',Xgb)
    ,('knn', knn)
    ,('lgbm',lgbm) 
    ,('sgd', sgd)
]

models2 = [
    ('svm', svc),
    ('poly',poly_svc)
    ,('Xgb',Xgb)
    ,('knn', knn)
    ,('lgbm',lgbm) 
    ,('sgd', sgd)
]

models3 = [
    ('sgd1', sgd),
    ('svm', svc)
    ,('Xgb',Xgb)
    ,('lgbm',lgbm)
]



In [16]:
from sklearn.ensemble import VotingClassifier

In [17]:
soft_vote  = VotingClassifier(models, voting='soft')
soft_vote.fit(X_train_, Y_train_over);

C:\Users\2469l\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\2469l\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:15:43] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[03:15:43] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [18]:
soft_vote2  = VotingClassifier(models2, voting='soft')
soft_vote2.fit(X_train_, Y_train_over);

C:\Users\2469l\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\2469l\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:16:15] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[03:16:15] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [19]:
soft_vote3  = VotingClassifier(models3, voting='soft')
soft_vote3.fit(X_train_, Y_train_over);

C:\Users\2469l\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:16:42] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[03:16:42] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [20]:
models_ = [
    ('sv1', soft_vote),
    ('sv2', soft_vote2),
    ('sv3', soft_vote3)
]
from sklearn.ensemble import VotingClassifier

soft_vote  = VotingClassifier(models_, voting='soft')
soft_vote.fit(X_train_, Y_train_over)
prediction=soft_vote.predict(X_test2);

C:\Users\2469l\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\2469l\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:17:15] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[03:17:15] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\2469l\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\2469l\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:17:47] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[03:17:47] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\2469l\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:18:15] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[03:18:15] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
